In [10]:
import pandas as pd
import numpy as np
import time
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [11]:
vic_data = pd.read_csv('../../team_code/chieh/victoria.csv', parse_dates=[0])

In [12]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

CHARGE = -1
DISCHARGE = 1

In [13]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [14]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['decision'] = 0
    
    df = df.drop([0], axis=0) 
    
    return df

In [15]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'decision'] = CHARGE

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'decision'] = DISCHARGE

    return df

In [16]:
POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """
    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap



def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch



def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
    
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue



def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if(df.at[i,'decision'] == CHARGE):
        raw_power = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)
    if(df.at[i,'decision'] == DISCHARGE):
        raw_power = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
    return raw_power



def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df



def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

In [17]:
def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [18]:
# Highest Revenue using cross validation
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
vic_price = run_all(vic_data)

Total revenue in the dataset: 122330144
Total days in the dataset: 1322.0
Revenue per day: 92534.1482602118
Time Complexity for running the entire Algorithm 3: 75.81970405578613s


In [19]:
vic_price.head()

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,decision
1,2018-01-01 00:30:00,92.46,0,0,0,0,0,1
2,2018-01-01 01:00:00,87.62,0,0,0,0,0,1
3,2018-01-01 01:30:00,73.08,0,0,0,0,0,1
4,2018-01-01 02:00:00,70.18,0,0,0,0,0,1
5,2018-01-01 02:30:00,67.43,0,0,0,0,0,1


### Helper functions from tutor

In [5]:
import sys
sys.path.append("/Users/hesterlim/energy-market-and-battery-optimization-group-15/instructions") # path of helper functions
from check import * # import everything from check.py

In [8]:
# Create a battery object and charge it. Testing
battery1 = Battery(10)
battery1.charge(10, 30)

(<StatusCodes.NORMAL: 0>, 133.785)

In [21]:
# Copy the dataset 
submission = vic_price.copy()

In [25]:
# Rename columns for submission
submission = submission.rename(columns = {'Time (UTC+10)' : 'datetime', 'raw_power' : 'power', 'closing' : 'capacity'})

In [26]:
# Create df similar to submission.csv
submit_columns = ['datetime', 'power', 'capacity']
submission = submission[submit_columns]

In [27]:
submission.head()

,datetime,power,capacity
1,2018-01-01 00:30:00,0,0
2,2018-01-01 01:00:00,0,0
3,2018-01-01 01:30:00,0,0
4,2018-01-01 02:00:00,0,0
5,2018-01-01 02:30:00,0,0


In [31]:
# Reset index to starting withn 0
submission = submission.reset_index(drop=True)

In [34]:
# Check submission
checked_df = check_submission(submission)

In [38]:
checked_df.head()

,datetime,flag
0,2018-01-01 00:30:00,0
1,2018-01-01 01:00:00,0
2,2018-01-01 01:30:00,0
3,2018-01-01 02:00:00,0
4,2018-01-01 02:30:00,0


In [45]:
# Not Normal. Flagged as bug
checked_df[checked_df['flag'] != 0]

,datetime,flag
34,2018-01-01 17:30:00,2
74,2018-01-02 13:30:00,2
85,2018-01-02 19:00:00,2
125,2018-01-03 15:00:00,2
166,2018-01-04 11:30:00,2
179,2018-01-04 18:00:00,2
183,2018-01-04 20:00:00,2
228,2018-01-05 18:30:00,2
276,2018-01-06 18:30:00,2
323,2018-01-07 18:00:00,2
